In [1]:
from dataset import *
from torch.utils.data import DataLoader

import torch
from torch import nn
import torch.nn.functional as F
from general_transformer import GenearalTransformer, Highway
from tqdm import tqdm
from transformers import AdamW, get_linear_schedule_with_warmup
from rlscore.measure.cindex_measure import cindex
from transformer_pytorch import FeedForward


In [2]:
def mean_max(x):
    return torch.mean(x, dim=2), torch.max(x, dim=2)[0]


class FGM():
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=0.01, emb_name='encoder.embedding'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0 and not torch.isnan(norm):
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='encoder.embedding'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name: 
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

        
class CNNSEBlock(nn.Module):
    def __init__(self, in_planes, planes, kernel_size=3, stride=1, padding=1):
        super(CNNSEBlock, self).__init__()

        #the first
        self.bn1 = nn.BatchNorm1d(planes)
        self.conv1 = nn.Conv1d(in_planes, planes, kernel_size=kernel_size, padding=padding, stride=stride, bias=False)

 
        # the first SE layers
        self.fc1 = nn.Conv1d(planes, planes//4, kernel_size=1)  # Use nn.Conv1d instead of nn.Linear
        self.fc2 = nn.Conv1d(planes//4, planes, kernel_size=1)


 

    def forward(self, x):
        
        out = self.conv1(x)
        out = F.relu(out)
        
#         # Squeeze
#         w = F.avg_pool1d(out, kernel_size=out.size(2))
#         w = F.relu(self.fc1(w))
#         w = torch.sigmoid(self.fc2(w))
#         # Excitation
#         out = out * w  
        
 
        return out

    
class TransformerDTA(nn.Module):
    def __init__(self, model_drug_atom, model_prot_atom,model_drug_token, model_prot_token, d_model, reduction='mean'):
        super(TransformerDTA, self).__init__()
        self.model_drug_atom = model_drug_atom
        self.model_prot_atom = model_prot_atom
        self.model_drug_token = model_drug_token
        self.model_prot_token = model_prot_token
        
#         self.classifier = nn.Linear(4 * d_model, 1)
        assert reduction in ['mean', 'cls', 'max'], 'Invalid reduction mode'
        self.reduction = reduction
        
 
        
        self.layernorms =  nn.LayerNorm( 4 * d_model * 3, eps=1e-6) 

        
        self.linear_predict = nn.Sequential(
            nn.Linear( 4 * d_model * 3, 1024),
#             nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 1024),
#             nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
#             nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1),
        )
        

        # activation and regularization
        dropout=0.2
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.highway = Highway(4, 4 * d_model * 3)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        
        self.prot_cnn_atom1 = CNNSEBlock(d_model, d_model*2)
        self.smi_cnn_atom1 = CNNSEBlock(d_model, d_model*2)
        
        self.prot_cnn_atom2 = CNNSEBlock(d_model*2, d_model*2)
        self.smi_cnn_atom2 = CNNSEBlock(d_model*2, d_model*2)
        
        self.prot_cnn_atom3 = CNNSEBlock(d_model*2, d_model*3)
        self.smi_cnn_atom3 = CNNSEBlock(d_model*2, d_model*3)
        
        
        
        self.prot_cnn_token1 = CNNSEBlock(d_model, d_model*2)
        self.smi_cnn_token1 = CNNSEBlock(d_model, d_model*2)
        
        self.prot_cnn_token2 = CNNSEBlock(d_model*2, d_model*2)
        self.smi_cnn_token2 = CNNSEBlock(d_model*2, d_model*2)
        
        self.prot_cnn_token3 = CNNSEBlock(d_model*2, d_model*3)
        self.smi_cnn_token3 = CNNSEBlock(d_model*2, d_model*3)
 
        
    def _embed(self, x, mask):
        if self.reduction == 'mean':
            x = x.masked_fill(~mask[..., None], 0).sum(1) / mask.sum(1)[:, None]
        elif self.reduction == 'max':
            x = x.masked_fill(~mask[..., None], 0).max(1)[0]
        else:
            x = x[:, 0, :]
        return x 
    
    
   
        
    def forward(self, drug_atom, drug_atom_mask,
                      prot_atom, prot_atom_mask,
                      drug_token, drug_token_mask,
                      prot_token, prot_token_mask):
        
         
        
        drug_atom_x = self.model_drug_atom(drug_atom, drug_atom_mask) 
        drug_atom_x = drug_atom_x.transpose(1,2)
        
        drug_atom_x = self.smi_cnn_atom1(drug_atom_x)
        drug_atom_x = self.smi_cnn_atom2(drug_atom_x)
        drug_atom_x = self.smi_cnn_atom3(drug_atom_x)
        drug_atom_x = torch.max(drug_atom_x, dim=2)[0]
        
        
        prot_atom_x = self.model_prot_atom(prot_atom, prot_atom_mask) 
        prot_atom_x = prot_atom_x.transpose(1,2)
        prot_atom_x = self.prot_cnn_atom1(prot_atom_x)
        prot_atom_x = self.prot_cnn_atom2(prot_atom_x)
        prot_atom_x = self.prot_cnn_atom3(prot_atom_x)
        prot_atom_x = torch.max(prot_atom_x, dim=2)[0]
        
        
        
        prot_token_x = self.model_prot_token(prot_token, prot_token_mask) 
        prot_token_x = prot_token_x.transpose(1,2)
        prot_token_x = self.prot_cnn_token1(prot_token_x)
        prot_token_x = self.prot_cnn_token2(prot_token_x)
        prot_token_x = self.prot_cnn_token3(prot_token_x)
        
        prot_token_x = torch.max(prot_token_x, dim=2)[0]

        
        
        
        drug_token_x = self.model_drug_token(drug_token, drug_token_mask) 
        drug_token_x = drug_token_x.transpose(1,2)
        drug_token_x = self.smi_cnn_token1(drug_token_x)
        drug_token_x = self.smi_cnn_token2(drug_token_x)
        drug_token_x = self.smi_cnn_token3(drug_token_x)
        drug_token_x = torch.max(drug_token_x, dim=2)[0]


        
       
        embedding = torch.cat([drug_atom_x, prot_atom_x,prot_token_x, drug_token_x], axis=-1)

#         embedding = torch.cat([drug_atom_x, drug_token_x, prot_atom_x, prot_token_x], axis=-1)
        embedding = self.layernorms(embedding)
        embedding = self.highway(embedding)
        
        out = self.linear_predict(embedding)
 
        return out
    

In [3]:
from sklearn.metrics import average_precision_score,auc,precision_recall_curve


def get_aupr(Y, P, threshold=7.0):
    # print(Y.shape,P.shape)
    Y = np.where(Y >= threshold, 1, 0)
    P = np.where(P >= threshold, 1, 0)
    aupr = average_precision_score(Y, P)
    precision, recall, thresholds = precision_recall_curve(Y, P)
    # Use AUC function to calculate the area under the curve of precision recall curve
    auc_precision_recall = auc(recall, precision)
 
    return auc_precision_recall


def r_squared_error(y_obs, y_pred):
    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)
    y_obs_mean = [np.mean(y_obs) for y in y_obs]
    y_pred_mean = [np.mean(y_pred) for y in y_pred]

    mult = sum((y_pred - y_pred_mean) * (y_obs - y_obs_mean))
    mult = mult * mult

    y_obs_sq = sum((y_obs - y_obs_mean) * (y_obs - y_obs_mean))
    y_pred_sq = sum((y_pred - y_pred_mean) * (y_pred - y_pred_mean))

    return mult / float(y_obs_sq * y_pred_sq)


def get_k(y_obs, y_pred):
    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)

    return sum(y_obs * y_pred) / float(sum(y_pred * y_pred))


def squared_error_zero(y_obs, y_pred):
    k = get_k(y_obs, y_pred)

    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)
    y_obs_mean = [np.mean(y_obs) for y in y_obs]
    upp = sum((y_obs - (k * y_pred)) * (y_obs - (k * y_pred)))
    down = sum((y_obs - y_obs_mean) * (y_obs - y_obs_mean))

    return 1 - (upp / float(down))


def get_rm2(ys_orig, ys_line):
    r2 = r_squared_error(ys_orig, ys_line)
    r02 = squared_error_zero(ys_orig, ys_line)

    return r2 * (1 - np.sqrt(np.absolute((r2 * r2) - (r02 * r02))))


def get_mse(y, f):
    mse = ((y - f) ** 2).mean(axis=0)
    return mse

def get_pearson(y, f):
    rp = np.corrcoef(y, f)[0, 1]
    return rp

def get_cindex(Y, P):
    summ = 0
    pair = 0

    for i in range(1, len(Y)):
        for j in range(0, i):
            if i is not j:
                if (Y[i] > Y[j]):
                    pair += 1
                    summ += 1 * (P[i] > P[j]) + 0.5 * (P[i] == P[j])

    if pair is not 0:
        return summ / pair
    else:
        return 0


In [4]:
def train_valid(model, train_loader, val_loader, EPOCHS, criterion, optimizer,scheduler, model_file_name):
    best_ci = 0
    early_stopping = EarlyStopping(patience=25, verbose=True, path=model_file_name)
    
    
    for epoch in range(1, EPOCHS):
        model.train()
        losses = []
        for drug_atom, target_atom, drug_token, target_token, affinity in tqdm(train_loader):
            optimizer.zero_grad()
            
            drug_atom, target_atom, drug_token, target_token, affinity = (x.to(device) for x in (drug_atom, target_atom, drug_token, target_token, affinity))

            drug_atom_mask, target_atom_mask, drug_token_mask, target_token_mask  = drug_atom != 0, target_atom != 0,drug_token != 0, target_token != 0
            
#             drug_atom_mask, target_atom_mask, drug_token_mask, target_token_mask = (x.to(device) for x in (drug_atom, target_atom, drug_token, target_token, affinity))

            
            
            logits = model(drug_atom, drug_atom_mask, target_atom, target_atom_mask,
                      drug_token, drug_token_mask,
                      target_token, target_token_mask)
            
            loss = criterion(logits, affinity)
            losses.append(loss.item())

             
            loss.backward()
#             torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
#             scheduler.step()

 



        print('Average loss', np.mean(losses))
        model.eval()
        with torch.no_grad():
            preds =  torch.Tensor()
            trues = torch.Tensor()



            for drug_atom, target_atom, drug_token, target_token, affinity in tqdm(val_loader):
                drug_atom, target_atom, drug_token, target_token, affinity = (x.to(device) for x in (drug_atom, target_atom, drug_token, target_token, affinity))

                drug_atom_mask, target_atom_mask, drug_token_mask, target_token_mask  = drug_atom != 0, target_atom != 0,drug_token != 0, target_token != 0

                logits = model(drug_atom, drug_atom_mask, target_atom, target_atom_mask,
                      drug_token, drug_token_mask,
                      target_token, target_token_mask)



                preds = torch.cat((preds, logits.cpu()), 0)
                trues = torch.cat((trues, affinity.view(-1, 1).cpu()), 0)

            preds, trues = preds.numpy().flatten(), trues.numpy().flatten()
            rm2 = get_rm2(trues, preds)   
            mse = get_mse(trues, preds)
            pearson = get_pearson(trues, preds)
            aupr = get_aupr(trues, preds, threshold=threshold)
            #ci_val = get_cindex(trues, preds)
            ci_val = cindex(np.float64(trues), np.float64(preds))
            early_stopping(ci_val, model)
    
            if early_stopping.early_stop:
                print("Early stopping")
                break
              
            print('val mse:', mse)
            print('val pearson', pearson)
            
            print('val AUPR', aupr)
            print('val ci:', ci_val)
            print(f'Epoch: {epoch + 1}')
            #print(cindex(np.float64(trues), np.float64(preds)))
#             if ci_val > best_ci:
#                 best_ci = ci_val  
#                 torch.save(model.state_dict(), model_file_name)

                
                
def predict(model, test_loader, threshold):
    model.eval()
    with torch.no_grad():
        preds =  torch.Tensor()
        trues = torch.Tensor()
        
     

        for drug_atom, target_atom, drug_token, target_token, affinity in tqdm(test_loader):
            drug_atom, target_atom, drug_token, target_token, affinity = (x.to(device) for x in (drug_atom, target_atom, drug_token, target_token, affinity))

            drug_atom_mask, target_atom_mask, drug_token_mask, target_token_mask  = drug_atom != 0, target_atom != 0,drug_token != 0, target_token != 0

            logits = model(drug_atom, drug_atom_mask, target_atom, target_atom_mask,
                  drug_token, drug_token_mask,
                  target_token, target_token_mask)

            
            
            preds = torch.cat((preds, logits.cpu()), 0)
            trues = torch.cat((trues, affinity.view(-1, 1).cpu()), 0)
            
        preds, trues = preds.numpy(), trues.numpy() 
        rm2 = get_rm2(trues.flatten(), preds.flatten())   
        mse = get_mse(trues.flatten(), preds.flatten())
        pearson = get_pearson(trues.flatten(), preds.flatten())
        aupr = get_aupr(trues.flatten(), preds.flatten(), threshold=threshold)
         
        #ci_val = get_cindex(trues.flatten(), preds.flatten())
        ci_val = cindex(np.float64(trues), np.float64(preds))
        print('-------------------test results---------------------------------')
        print('val mse:', mse)
        print('val pearson', pearson)

        print('val AUPR', aupr)
        print('val ci:', ci_val)
        print('-------------------test results---------------------------------')
    return rm2, mse, pearson,aupr,ci_val





In [5]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score > self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [6]:
dataset = 'BKI'

if dataset == 'KIBA_S1': 
    data_file = '/home/junjie/MultiViewDTA/data/kiba-subset1-data.csv'
    threshold = 12.1
elif dataset == 'KIBA_S2': 
    data_file = '/home/junjie/MultiViewDTA/data/kiba-subset2-data.csv'
    threshold = 12.1
elif dataset == 'KIBA_U1':
    data_file = '/home/junjie/MultiViewDTA/data/kiba-uniq-data.csv'
    threshold = 12.1
elif dataset == 'metz':
    data_file = '/home/junjie/MultiViewDTA/data/metz-data.csv'
elif dataset == 'BKD':
    data_file = '/home/junjie/MultiViewDTA/data/BindingDB_Kd-uniq-data.csv'
    threshold = 7.6
elif dataset == 'BKI':
    data_file = '/home/junjie/MultiViewDTA/data/BindingDB-uniq-data.csv'
    threshold = 7.6
elif dataset == 'BKI_S1':
    data_file = '/home/junjie/MultiViewDTA/data/BindingDB-subset1-data.csv'
    threshold = 7.6
elif dataset == 'davis':
    data_file = '/home/junjie/MultiViewDTA/data/davis-uniq-data.csv'
    threshold = 7.0
else:
    sys.exit("Unknown Dataset") 


all_drug = []
all_protein = []
all_Y = []
with open(data_file, 'r') as f:
    all_lines = f.readlines()

    for line in all_lines:
        row = line.rstrip().split(',')
        all_drug.append(row[0])
        all_protein.append(row[1])
        all_Y.append(float(row[2]))

kf = KFold(n_splits=5, shuffle=True)
# batch_size = 256
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
MODEL_TYPE = 'transformer'

EPOCHS = 350
BATCH_SIZE = 128

VOCAB_SIZE_drug_atom = 67+1
VOCAB_SIZE_prot_atom = 25 + 1
# VOCAB_SIZE_drug_token = 23532+ 1
# VOCAB_SIZE_prot_atom = 16693+ 1
VOCAB_SIZE_drug_token = 1000+ 1
VOCAB_SIZE_prot_token = 10000+ 1


drug_len_atom = 256
prot_len_atom = 1024
drug_len_token = 128
prot_len_token = 512




    
    

In [7]:
def run_grid_cv_4_model(lr, n_layer, n_head, d_model ):
    N_HEADS = n_head
    N_LAYERS = n_layer
    D_MODEL = d_model
    
    
    all_loss =  np.zeros((5,1))
    all_ci =  np.zeros((5,1))
    all_ci2 = np.zeros((5,1))
    all_mse2 = np.zeros((5,1))+10
    all_r = np.zeros((5,1))
    all_aupr = np.zeros((5,1))
    all_rm2 = np.zeros((5,1))




    for split, ( train_index, test_index) in enumerate( kf.split(all_Y)):


        train_valid_protein_cv = np.array(all_protein)[train_index]
        train_valid_drug_cv = np.array(all_drug)[train_index]

        train_valid_Y_cv = np.array(all_Y)[train_index]

        test_protein_cv = np.array(all_protein)[test_index]


        test_drug_cv = np.array(all_drug)[test_index]

        test_Y_cv = np.array(all_Y)[test_index]

        train_size = train_valid_protein_cv.shape[0] 

        valid_size =  int(train_size/5.0) #?


        train_protein_cv = train_valid_protein_cv[:train_size-valid_size]
        train_drug_cv = train_valid_drug_cv[:train_size-valid_size]
        train_Y_cv = train_valid_Y_cv[:train_size-valid_size]

        valid_protein_cv = train_valid_protein_cv[train_size-valid_size:]
        valid_drug_cv = train_valid_drug_cv[train_size-valid_size:]
        valid_Y_cv = train_valid_Y_cv[train_size-valid_size:]


        train_ds = pack_dataset2(train_drug_cv, train_protein_cv, train_Y_cv)
        valid_ds = pack_dataset2(valid_drug_cv, valid_protein_cv, valid_Y_cv)
        test_ds = pack_dataset2(test_drug_cv, test_protein_cv, test_Y_cv)

        train_loader = DataLoader(train_ds, batch_size= BATCH_SIZE, shuffle=True, num_workers=1)
        test_loader = DataLoader(test_ds, batch_size= BATCH_SIZE, shuffle=True, num_workers=1)
        valid_loader = DataLoader(valid_ds, batch_size= BATCH_SIZE, shuffle=True, num_workers=1)

        drug_atom_transformer = GenearalTransformer(MODEL_TYPE, VOCAB_SIZE_drug_atom, drug_len_atom, D_MODEL, N_HEADS, N_LAYERS,device)
        drug_token_transformer = GenearalTransformer(MODEL_TYPE, VOCAB_SIZE_drug_token, drug_len_token, D_MODEL, N_HEADS, N_LAYERS,device)
        prot_atom_transformer = GenearalTransformer(MODEL_TYPE, VOCAB_SIZE_prot_atom, prot_len_atom, D_MODEL, N_HEADS, N_LAYERS,device)
        prot_token_transformer = GenearalTransformer(MODEL_TYPE, VOCAB_SIZE_prot_token, prot_len_token, D_MODEL, N_HEADS, N_LAYERS,device)

        drug_atom_transformer = drug_atom_transformer.to(device)

        drug_token_transformer = drug_token_transformer.to(device)
        prot_atom_transformer = prot_atom_transformer.to(device)
        prot_token_transformer = prot_token_transformer.to(device)

        reduction = 'max'
#         model_file_name = 'models/spm-'+ dataset+'-'+MODEL_TYPE + '-'+reduction +'.model'
#         model_file_name = 'models/spm-nhead-'+str(N_HEADS)+'-nlayer-'+str(N_LAYERS)+'-d-model-'+str(D_MODEL)+ dataset+'-'+MODEL_TYPE + '-'+reduction +'.model'
        model_file_name = 'models/spm-nhead-'+str(N_HEADS)+'-nlayer-'+str(N_LAYERS)+'-d-model-'+str(D_MODEL)+ dataset+'-'+MODEL_TYPE + '-lr-'+str(lr) +'.model'


        model = TransformerDTA(drug_atom_transformer, prot_atom_transformer, drug_token_transformer,prot_token_transformer,
                               D_MODEL, reduction=reduction)
        
        model = nn.DataParallel(model, device_ids=[0,2])
        model.to(device)
    #     optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
    #     optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.01)
        optimizer = AdamW(model.parameters(), lr=lr,  eps=1e-8)
        criterion =  nn.MSELoss()

        scheduler = get_linear_schedule_with_warmup(
            optimizer, 
            num_warmup_steps=0, 
            num_training_steps=EPOCHS
        )

        train_valid(model, train_loader, valid_loader, EPOCHS, criterion,optimizer,scheduler, model_file_name)

        model.load_state_dict(torch.load(model_file_name))

        rm2, mse, pearson,aupr,ci_val = predict(model, test_loader, threshold)
        

        all_mse2[split] = mse
        all_r[split] = pearson
        all_aupr[split] = aupr
        all_rm2[split] = rm2
        all_ci2[split] = ci_val
    return  all_mse2, all_r, all_aupr, all_rm2, all_ci2
    

In [8]:
lr = 1e-4
n_layer = 1
n_head  = 4
d_model = 128 

all_mse2, all_r, all_aupr, all_rm2, all_ci2 = run_grid_cv_4_model(lr, n_layer, n_head, d_model )

                
             

  0%|          | 0/162 [00:00<?, ?it/s]

Average loss 2.4237745099761536


100%|██████████| 162/162 [00:14<00:00, 11.56it/s]


Validation loss decreased (inf --> 0.685424).  Saving model ...


  0%|          | 0/646 [00:00<?, ?it/s]

...

Average loss 0.02293854557560186


  0%|          | 0/202 [00:00<?, ?it/s]

EarlyStopping counter: 25 out of 25
Early stopping


100%|██████████| 202/202 [00:16<00:00, 11.99it/s]


-------------------test results---------------------------------
val mse: 0.82715815
val pearson 0.8182450564017401
val AUPR 0.7861844520455965
val ci: 0.8141443555923376
-------------------test results---------------------------------


In [9]:
print('*='*20)
print('cindex:  {0:6f}({1:6f})'.format(np.mean(all_ci2),  np.std(all_ci2)))
print('mse:     {0:6f}({1:6f})'.format(np.mean(all_mse2), np.std(all_mse2)))
print('rm2:     {0:6f}({1:6f})'.format(np.mean(all_rm2),  np.std(all_rm2)))
print('pearson: {0:6f}({1:6f})'.format(np.mean(all_r),    np.std(all_r)))
print('AUPR:    {0:6f}({1:6f})'.format(np.mean(all_aupr), np.std(all_aupr)))
print()


*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
cindex:  0.814497(0.002002)
mse:     0.815913(0.011410)
rm2:     0.609317(0.008438)
pearson: 0.819709(0.002366)
AUPR:    0.784454(0.002604)

